# Описалово

# 0. Подготовительные шаги и функции  
Загружаю библиотеки и объявляю константы

In [53]:
import pandas as pd
from scipy import stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from statistics import mean 
import math
DF_NAMES = ['df_cow', 'df_main', 'df_dad']

# 1. Предобработка данных

## Преамбула  
### Для выполнения задачи заказчиком были переданы три набора данных  
 - ferma_main.csv - содержит данные о стаде на данный момент;  
 - ferma_dad.csv - содержит клички быков родителей каждой коровы в стаде фермера;  
 - cow_buy.csv - данне о коровах "ЭкоФермы" предложенных к покупке заказчиком;  
 В ходе дальнейших действий я произведу манипуляции с данными подготавливая их для дальнейших операций.

#### Загружаю данные из файлов в датафреймы. Т.к. работу выполнял локально добавил проверку.

In [54]:
try:
    df_cow=pd.read_csv('c:/ps/project/happy_farm/data/cow_buy.csv', sep=';', decimal=",")
    df_main=pd.read_csv('c:/ps/project/happy_farm/data/ferma_main.csv', sep=';', decimal=",")
    df_dad=pd.read_csv('c:/ps/project/happy_farm/data/ferma_dad.csv', sep=';', decimal=",")
except:
    df_cow=pd.read_csv('/datasets/cow_buy.csv', sep=';', decimal=",")
    df_main=pd.read_csv('/datasets/ferma_main.csv', sep=';', decimal=",")
    df_dad=pd.read_csv('/datasets/ferma_dad.csv', sep=';', decimal=",")

В файлах использовался не стандартный разделитель, пришлось добавить разделитель. Указал разделитель для дробных данных, чтобы иметь возможность их корректно обрабатывать.

#### Проанализирую содержимое таблиц и тип хранимых данных.

##### Выведу пять случайных строк каждой таблицы, чтобы посмотреть на их содержимое.

In [55]:
display(df_cow.sample(5))
display(df_main.sample(5))
display(df_dad.sample(5))

,Порода,Тип пастбища,порода папы_быка,Имя_папы,"Текущая_жирность,%","Текущий_уровень_белок,%",Возраст
10,Вис Бик Айдиал,холмистое,Соверин,Соловчик,3.73,3.055,более_2_лет
2,РефлешнСоверинг,равнинное,Соверин,Барин,3.59,3.074,более_2_лет
9,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.079,менее_2_лет
12,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,3.68,3.046,более_2_лет
15,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.065,более_2_лет


,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст
620,621,6813,14.5,1712,0.96,РефлешнСоверинг,Равнинное,Айдиал,3.72,3.075,вкусно,более_2_лет
397,398,5207,13.3,1829,0.89,РефлешнСоверинг,Холмистое,Соверин,3.48,3.077,не вкусно,менее_2_лет
95,96,6976,16.4,2263,0.95,Вис Бик Айдиал,Холмистое,Айдиалл,3.71,3.078,вкусно,более_2_лет
316,317,6515,15.2,2221,0.93,Вис Бик Айдиал,Холмистое,Соверин,3.72,3.077,не вкусно,более_2_лет
401,402,5814,14.6,1794,0.89,Вис Бик Айдиал,Холмистое,Соверин,3.59,3.077,вкусно,более_2_лет


,id,Имя Папы
476,477,Барин
546,547,Барин
76,77,Геркулес
459,460,Соловчик
176,177,Геркулес


##### Посмотрю соответствие типов столбцов описанным в задаче типам.

In [56]:
display(df_cow.info(True))
display(df_main.info(True))
display(df_dad.info(True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Порода                   20 non-null     object 
 1   Тип пастбища             20 non-null     object 
 2   порода папы_быка         20 non-null     object 
 3   Имя_папы                 20 non-null     object 
 4   Текущая_жирность,%       20 non-null     float64
 5   Текущий_уровень_белок,%  20 non-null     float64
 6   Возраст                  20 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     634 non-null    int64  
 1   Удой, кг                               634 non-null    int64  
 2   ЭКЕ (Энергетическая кормовая единица)  634 non-null    float64
 3   Сырой протеин, г                       634 non-null    int64  
 4   СПО (Сахаро-протеиновое соотношение)   634 non-null    float64
 5   Порода                                 634 non-null    object 
 6   Тип пастбища                           634 non-null    object 
 7   порода папы_быка                       634 non-null    object 
 8   Жирность,%                             634 non-null    float64
 9   Белок,%                                634 non-null    float64
 10  Вкус молока                            634 non-null    object 
 11  Возрас

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        629 non-null    int64 
 1   Имя Папы  629 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.0+ KB


None

Можно заметить, что пропусков в столбцах нет.  
Формат столбцов не соответствует хранимым в них данным. Именование столбцов тоже мне не подходят. Изменю тип даннх столбцов немного позже - после переименования столбцов.

#### Проверю именование столбцов

In [57]:
display('COW - table')
display(df_cow.columns.tolist())
display('MAIN - table')
display(df_main.columns.tolist())
display('DAD - table')
display(df_dad.columns.tolist())

'COW - table'

['Порода',
 'Тип пастбища',
 'порода папы_быка',
 'Имя_папы',
 'Текущая_жирность,%',
 'Текущий_уровень_белок,%',
 'Возраст']

'MAIN - table'

['id',
 'Удой, кг',
 'ЭКЕ (Энергетическая кормовая единица)',
 'Сырой протеин, г',
 'СПО (Сахаро-протеиновое соотношение)',
 'Порода',
 'Тип пастбища',
 'порода папы_быка',
 'Жирность,%',
 'Белок,%',
 'Вкус молока',
 'Возраст']

'DAD - table'

['id', 'Имя Папы']

Именование столбцов не соттветсвует требованиям. Переименовываю.

In [58]:
df_cow.columns = ['breed',
                  'pasture_type',
                  'breed_dad',
                  'dad_name',
                  'fat_per',
                  'protein_per',
                  'age']
df_main.columns = ['id',
                   'yield',
                   'eke',
                   'protein_weight',
                   'spo',
                   'breed',
                   'pasture_type',
                   'breed_dad',
                   'fat_per',
                   'protein_per',
                   'taste',
                   'age']
df_dad.columns = ['id', 
                  'name']



#### Меняю тип данных столбцов в соответствии с таблицей:  

**df_cow**
| Имя столбца    | Тип данных  | Новый тип данных |
|:---------------|:-----------:|:----------------:|
| breed          | object      | object           |
| pasture_type   | object      | object           |
| breed_dad      | object      | object           |
| dad_name       | object      | object           |
| fat_per        | object      | float64          |
| protein_per    | object      | float64          |
| age            | object      | object           |  

**df_main**
| Имя столбца    | Тип данных  | Новый тип данных |
|:---------------|:-----------:|:----------------:|
| id             | int64       | int64            |
| yield          | int64       | int64            |
| eke            | object      | float64          |
| protein_weight | int64       | int64            |
| spo            | object      | float64          |
| breed          | object      | object           |
| pasture_type   | object      | object           |
| breed_dad      | object      | object           |
| fat_per        | object      | float64          |
| protein_per    | object      | float64          |
| taste          | object      | object           |
| age            | object      | object           |  

**df_dad**
| Имя столбца    | Тип данных  | Новый тип данных |
|:---------------|:-----------:|:----------------:|
| id             | object      | int64            |
| name           | object      | object           |

In [59]:
df_cow = df_cow.astype({
    'fat_per': float,
    'protein_per': float
    
})

df_main = df_main.astype({
    'eke': float,
    'spo': float,
    'fat_per': float,
    'protein_per': float
})

df_dad = df_dad.astype({
    'id': float    
})

#### Проверю таблицы на дубликаты

##### Выведу список явных дубликатов

In [60]:
print(f'В df_cow найдено дубликатов: {df_cow.duplicated().sum()}')
print(f'В df_main найдено дубликатов: {df_main.duplicated().sum()}')
print(f'В df_dad найдено дубликатов: {df_dad.duplicated().sum()}')

В df_cow найдено дубликатов: 4
В df_main найдено дубликатов: 5
В df_dad найдено дубликатов: 0


##### Удаляю явные дубликаты

In [61]:
df_cow = df_cow.drop_duplicates()
df_main = df_main.drop_duplicates()

## Вывод:  
Я очистил, дополнил, исправил.

# 2. Исследовательский анализ данных

## Преамбула

## Вывод

# 3. Подготовка данных к машинному обучению

## Преамбула

## Вывод

# 4. Обучение и настройка модели

## Преамбула

## Вывод

# 5. Тестирование модели

## Преамбула

## Вывод

# 6. Вывод